In [24]:
#importing packages needed to perform Tasks
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np             #bringing in numpy libraries
import cv2                     #bringing in OpenCV libraries
import math
import os                      #bringing in OS operations libraries
# Import everything needed to edit/save/watch video clip
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
os.listdir("test_images/")
%matplotlib inline



# Function for Pipeline Below 
# The output returned is a color image (3 channel) of processed video with lines are drawn on lanes


def process_image(image):
    
    
    #Declare Variables needed to perform task
    right_side_x = []
    right_side_y = []
    left_side_x = []
    left_side_y = []
    right_gradients = []
    left_gradients = []

    
    # grayscale the image that has been taken in by function
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 5
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)

    # Define our parameters for Canny and apply
    low_threshold = 50
    high_threshold = 150
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

    # Next we'll create a masked edges image using cv2.fillPoly()
    mask = np.zeros_like(edges)   
    ignore_mask_color = 255   

    # Defining a four sided polygon to mask
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]), (460, 320), (490, 308), (imshape[1],imshape[0])]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)
    
    
    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 50     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 40 #minimum number of pixels making up a line
    max_line_gap = 100    # maximum gap in pixels between connectable line segments
    α = 0.8
    β = 1
    λ = 0
    line_image = np.copy(image)*0 # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),
                            min_line_length, max_line_gap)
    
    # Iterate over the output "lines" and draw lines on a blank image
    for line in lines:
        for x1,y1,x2,y2 in line:
            #cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),5)
            #cv2.line(line_image,(x1*2,y1*2),(x2*2,y2*2),(255,0,0),5)
            
            midpoint = imshape[1]/2
            bottom = imshape[0]

            a = (y2 - y1) / (x2 - x1)
            b = int(y1 - a * x1)
            
            #Separating left & Right Lane lines
            if a < 0 and x1 < midpoint and x2 < midpoint:  # Draw  left side lines
                left_side_y.append(y1)
                left_side_y.append(y2)
                left_side_x.append(x1)
                left_side_x.append(x2)
                left_gradients.append(a)
            elif a > 0 and x1 > midpoint and x2 > midpoint: # Draw Right side lines
                right_side_y.append(y1)
                right_side_y.append(y2)
                right_side_x.append(x1)
                right_side_x.append(x2)
                right_gradients.append(a)
               

    #right lane draw
    
    if right_side_y:
        min_right_side_y = min(right_side_y)
        right_ref = right_side_y.index(min_right_side_y)
        right_x1 = right_side_x[right_ref]
        right_y1 = right_side_y[right_ref]
        right_gradient = np.mean(right_gradients) 


        right_x2 = int(right_x1 + (bottom - right_y1) / right_gradient)

        #Final draw for each right line drawn    
        cv2.line(line_image, (right_x1, right_y1), (right_x2, bottom),(255,0,0),10)
  
    #left lane draw
    
    if left_side_y:
        min_left_side_y = min(left_side_y)
        left_ref = left_side_y.index(min_left_side_y)
        left_x1 = left_side_x[left_ref]
        left_y1 = left_side_y[left_ref]
        left_gradient = np.mean(left_gradients)        
        left_x2 = int(left_x1 + (bottom - left_y1) / left_gradient)
        
        #Final draw for each left line drawn
        cv2.line(line_image, (left_x1, left_y1), (left_x2, bottom),(255,0,0),10)
    
    # Draw the lines on the edge image
    result = cv2.addWeighted(image, α, line_image, β, λ) 
   
    #Ouput test images for write up
    cv2.imwrite('test_images_output/grayscale.jpg',gray)
    cv2.imwrite('test_images_output/blur_gray.jpg',blur_gray)
    cv2.imwrite('test_images_output/edges.jpg',edges)
    cv2.imwrite('test_images_output/m_edges.jpg',masked_edges)
    cv2.imwrite('test_images_output/result.jpg',result)
    
    #exit function and return result  
    return result


#Code that calls function to process image/video and output to file
white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image)
white_clip.write_videofile(white_output, audio=False)


[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:24<00:00,  8.39it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 



In [ ]:
2
3
